In [ ]:
%pip install chess tensorflow keras

In [1]:
import chess
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import optimizers, losses

# Define the evaluation function model
def create_evaluation_model():
    model = keras.Sequential([
        #input layer
        keras.layers.Dense(132, activation='relu', input_shape=[132], kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        
        #hidden layers
        #keras.layers.Dense(4096, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dropout(0.2),
        #keras.layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        #keras.layers.Dropout(0.2),
        #keras.layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        #keras.layers.Dropout(0.2),

        #output layer
        keras.layers.Dense(1, activation='linear')
    ])
    # Set the laerning rate
    learning_rate = 0.01
    
    # Choose the optimizer
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    #optimizer = optimizers.RMSprop(learning_rate=learning_rate)
    #optimizer = optimizers.SGD(learning_rate=learning_rate)
    #optimizer = optimizers.Adagrad(learning_rate=learning_rate)
    
    # Choose the loss function
    #loss=losses.mean_absolute_error
    loss=losses.mean_squared_error
    #loss=losses.binary_crossentropy
    
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [2]:
# Convert FEN to a feature vector
def fen_to_features(fen):
    board = chess.Board(fen)
    features = np.zeros(132)  # Increase the feature vector size to accommodate additional features
    
    for i in range(64):
        piece = board.piece_at(i)
        if piece is not None:
            piece_type = piece.piece_type
            color = piece.color
            sign = 1 if color == chess.WHITE else -1
            features[i] = sign * piece_type
    
    # Additional features
    features[64] = int(board.turn)  # Current player's turn (0 for black, 1 for white)
    
   # Piece mobility
    for i in range(64):
        piece = board.piece_at(i)
        if piece is not None:
            piece_mobility = len(board.attacks(i))
            features[i + 65] = piece_mobility
    
    # King safety
    white_king_safety = calculate_king_safety(board, chess.WHITE)
    black_king_safety = calculate_king_safety(board, chess.BLACK)
    features[129] = white_king_safety
    features[130] = black_king_safety

    # Forced mate flag
    if board.is_checkmate():
        if board.turn == chess.WHITE and board.result() == "1-0":
            features[131] = 1.0
        elif board.turn == chess.BLACK and board.result() == "0-1":
            features[131] = -1.0
    
    return features

def calculate_king_safety(board, color):
    king_square = board.king(color)
    king_safety = 0
    
    # Example: Evaluate king safety by counting the number of squares attacked by opponent's pieces
    opponent_color = not color
    for i in range(64):
        if board.is_attacked_by(opponent_color, i):
            king_safety += 1
    
    return king_safety


In [7]:
def evaluate_position(model, position):
    eval_score = model.predict(np.array([position]))[0][0]
    return eval_score

def is_forced_mate(eval_score, color):
    if color == chess.WHITE and eval_score >= 4000:
        return True
    elif color == chess.BLACK and eval_score <= -4000:
        return True
    return False

# Read training data from CSV
training_data = pd.read_csv('C:\Desktop\IITB_SoC_Deep_Carlsen\Training Data\chessData-10L.csv')[0:400001]

# Convert training data to feature vectors
training_positions = []
training_scores = []

for i in range(len(training_data['FEN'])):
    fen = training_data['FEN'][i]
    position = fen_to_features(fen)
    score = float(training_data['eval'][i])
    
    if not is_forced_mate(score, chess.WHITE) and not is_forced_mate(score, chess.BLACK):
        training_positions.append(position)
        training_scores.append(score)

In [8]:
# Create and train the evaluation function model
model = create_evaluation_model()

# Define the ProgbarLogger callback
#progbar_logger = keras.callbacks.ProgbarLogger()

# Fit the model with the ProgbarLogger callback
#model.fit(np.array(training_positions), np.array(training_scores), epochs=500, batch_size=1048575, callbacks=[progbar_logger])
model.fit(np.array(training_positions), np.array(training_scores), epochs=10, batch_size=210)

Epoch 1/10
1888/1888 [==============================] - 109s 55ms/step - loss: 34.3251
Epoch 2/10
1888/1888 [==============================] - 89s 47ms/step - loss: 10.2038
Epoch 3/10
1888/1888 [==============================] - 89s 47ms/step - loss: 8.1193
Epoch 4/10
1888/1888 [==============================] - 96s 51ms/step - loss: 7.1351
Epoch 5/10
1888/1888 [==============================] - 89s 47ms/step - loss: 6.8595
Epoch 6/10
1888/1888 [==============================] - 87s 46ms/step - loss: 6.8348
Epoch 7/10
1888/1888 [==============================] - 104s 55ms/step - loss: 6.8321
Epoch 8/10
1888/1888 [==============================] - 247s 131ms/step - loss: 6.8049
Epoch 9/10
1888/1888 [==============================] - 175s 93ms/step - loss: 6.8192
Epoch 10/10
1888/1888 [==============================] - 111s 59ms/step - loss: 6.7619


In [9]:
# Test example
test_example = 'r2q1rk1/p1p1bppp/2p2n2/3p2B1/4P1b1/2NB1Q2/PPP2PPP/2KR3R w - - 8 11'
# A position from a game played by me as black with eval = -1.3, although both side have equal values of pieces (material)

# Convert test example to feature vector
test_position = fen_to_features(test_example)

# Predict the evaluation score for the test example
evaluation_score = evaluate_position(model, test_position)

print('Test Example:', test_example)
print('Evaluation Score:', evaluation_score)

1/1 [==============================] - 0s 458ms/step
Test Example: r2q1rk1/p1p1bppp/2p2n2/3p2B1/4P1b1/2NB1Q2/PPP2PPP/2KR3R w - - 8 11
Evaluation Score: 0.05933802


In [13]:
# Function to generate the best move based on the given FEN ID and evaluation score
def generate_best_move(fen, eval_score, model, depth=2):
    board = chess.Board(fen)
    color = board.turn

    # Get all legal moves from the given position
    legal_moves = list(board.legal_moves)

    # Initialize the best move and its corresponding evaluation score
    best_move = None
    best_eval = float('-inf') if color == chess.WHITE else float('inf')

    # Iterate through each legal move
    for move in legal_moves:
        # Make the move on a temporary board
        temp_board = board.copy()
        temp_board.push(move)

        # Convert the temporary board to feature vector
        temp_features = fen_to_features(temp_board.fen())

        # Calculate the evaluation score for the temporary board
        temp_eval = model.predict(np.array([temp_features]))[0][0]

        # If it is the white player's turn, maximize the evaluation score
        if color == chess.WHITE:
            if temp_eval > best_eval:
                best_eval = temp_eval
                best_move = move
        # If it is the black player's turn, minimize the evaluation score
        else:
            if temp_eval < best_eval:
                best_eval = temp_eval
                best_move = move

    return best_move

# Usage example
# Assuming 'model' is the trained evaluation function model
# fen_id = 'r1bqk2r/pppp1ppp/2nb4/4p3/3PP3/2N5/PPP2PPP/R1BQKBNR w KQkq - 1 7'
# eval_score = 0.5  # Replace with the actual evaluation score from the evaluation function
best_move = generate_best_move(test_example, evaluation_score, model)

print('Best Move:', best_move)


1/1 [==============================] - 0s 45ms/step
Best Move: g5h6


In [15]:
# save and load the model
model.save('DeepCarlsen.h5')
model = keras.models.load_model('DeepCarlsen.h5')